In [1]:
import pandas as pd
import numpy as np

# 加载数据集
file_path = 'movieMY.xlsx'  # 替换为您的文件路径
movie_data = pd.read_excel(file_path, sheet_name='Sheet1')

# 清洗'主/导演'列，去除'等'或'*'及其后的所有字符
movie_data['导演'] = movie_data['导演'].str.replace(r'等.*', '', regex=True)
movie_data['导演'] = movie_data['导演'].str.replace(r'\*.*', '', regex=True)
movie_data['主演'] = movie_data['主演'].str.replace(r'等.*', '', regex=True)
movie_data['主演'] = movie_data['主演'].str.replace(r'\*.*', '', regex=True)

# 显示清洗后的数据
print(movie_data)#.head()


     序号         片名    票房（亿）      上映年月日  地区  豆瓣评分                  导演  \
0     1        长津湖  57.5879 2021-09-30  中国   7.4                  吴京   
1     2        战狼2  56.9454 2017-07-27  中国   7.1                  吴京   
2     3     你好，李焕英  54.1308 2021-02-12  中国   7.8                  贾玲   
3     4    哪吒之魔童降世  50.3570 2019-07-26  中国   8.4                  饺子   
4     5       流浪地球  46.8645 2019-02-05  中国   7.9                  郭帆   
..  ...        ...      ...        ...  ..   ...                 ...   
95   96  哥斯拉2:怪兽之王   9.3740 2019-05-31  美国   6.3             迈克尔·道赫蒂   
96   97      超时空同居   9.0307 2018-05-18  中国   6.8                  苏伦   
97   98        老炮儿   9.0116 2015-12-24  中国   7.6                  管虎   
98   99   阿丽塔：战斗天使   8.9693 2019-02-22  美国   7.5  罗伯特·罗德里格兹、詹姆斯·卡梅隆    
99  100       绝地逃亡   8.8769 2016-07-21  中国   5.7               雷尼·哈林   

                 主演  
0           吴京、易烊千玺  
1                吴京  
2         贾玲、张小斐、沈腾  
3               NaN  
4        屈楚萧、吴京、李光洁  
.. 

In [3]:
create_table_movie_info = '''
Create Table movie_info
( 
 movie_id varchar(10) Not Null,
 movie_name varchar(20) Not Null,
 box_office decimal(10,4),
 release_date datetime,
 country varchar(20),
 douban_rating decimal(3,1),
 year int check( year>=1000 and  year<=2100),
 primary key(movie_id)
);
'''
# 生成INSERT语句
insert_movie_info_statements = [create_table_movie_info]
for index, row in movie_data.iterrows():
    movie_id = str(index + 1)  # 假设序号即为电影ID
    movie_name = row['片名']
    box_office = row['票房（亿）']
    release_date = row['上映年月日']
    country = row['地区']
    douban_rating = row['豆瓣评分']
    year = pd.to_datetime(release_date).year  # 提取年份

    insert_movie_info_sql = f"insert into movie_info values('{movie_id}','{movie_name}',{box_office},'{release_date}','{country}',{douban_rating},{year});"
    insert_movie_info_statements.append(insert_movie_info_sql)

insert_movie_info_statements

['\nCreate Table movie_info\n( \n movie_id varchar(10) Not Null,\n movie_name varchar(20) Not Null,\n box_office decimal(10,4),\n release_date datetime,\n country varchar(20),\n douban_rating decimal(3,1),\n year int check( year>=1000 and  year<=2100),\n primary key(movie_id)\n);\n',
 "insert into movie_info values('1','长津湖',57.5879,'2021-09-30 00:00:00','中国',7.4,2021);",
 "insert into movie_info values('2','战狼2',56.9454,'2017-07-27 00:00:00','中国',7.1,2017);",
 "insert into movie_info values('3','你好，李焕英',54.1308,'2021-02-12 00:00:00','中国',7.8,2021);",
 "insert into movie_info values('4','哪吒之魔童降世',50.357,'2019-07-26 00:00:00','中国',8.4,2019);",
 "insert into movie_info values('5','流浪地球',46.8645,'2019-02-05 00:00:00','中国',7.9,2019);",
 "insert into movie_info values('6','唐人街探案3',45.2341,'2021-02-12 00:00:00','中国',5.3,2021);",
 "insert into movie_info values('7','复仇者联盟4:终局之战',42.5013,'2019-04-24 00:00:00','美国',8.5,2019);",
 "insert into movie_info values('8','红海行动',36.5163,'2018-02-16 00:0

In [4]:
# # 确保导演和主演列都是字符串类型
# movie_data['导演'] = movie_data['导演'].fillna('').astype(str)
# movie_data['主演'] = movie_data['主演'].fillna('').astype(str)

# 分离导演和主演数据
directors = movie_data['导演'].str.split('、').explode().dropna().unique()
actors = movie_data['主演'].str.split('、').explode().dropna().unique()

# 合并导演和主演数据，移除重复项，并创建电影人ID
unique_movie_people = pd.Series(np.unique(np.concatenate((directors, actors))))
movie_people_df = pd.DataFrame({'电影人ID': range(1001, 1001 + len(unique_movie_people)), '电影人名称': unique_movie_people})

print(movie_people_df)

     电影人ID    电影人名称
0     1001  D.J.卡卢索
1     1002  F·加里·格雷
2     1003      乌尔善
3     1004   乔·德特杜巴
4     1005     乔·沃茨
..     ...      ...
206   1207      黄志忠
207   1208      黄晓明
208   1209      黄景瑜
209   1210       黄渤
210   1211       黄轩

[211 rows x 2 columns]


In [5]:
# 创建电影-电影人关系表
relationship_records = []

for index, row in movie_data.iterrows():
    movie_id = row['序号']
#     print(row)
    directors = row['导演'].split('、')if pd.notna(row['导演']) else []
    actors = row['主演'].split('、')if pd.notna(row['主演']) else []
#     print(directors,actors)
    # 添加导演关系
    for director in directors:
        if director in unique_movie_people.values and director != '':  # 忽略空字符串或未知导演
            director_id = movie_people_df[movie_people_df['电影人名称'] == director]['电影人ID'].iloc[0]
            relationship_records.append({'电影人ID': director_id, '电影序号': movie_id, '关系': '导演'})

    # 添加主演关系
    for actor in actors:
        if actor in unique_movie_people.values and actor != '':  # 忽略空字符串或未知演员
            actor_id = movie_people_df[movie_people_df['电影人名称'] == actor]['电影人ID'].iloc[0]
            relationship_records.append({'电影人ID': actor_id, '电影序号': movie_id, '关系': '主演'})

movie_people_relationship_df = pd.DataFrame(relationship_records)

# 显示结果
# print(movie_people_df.head())
print(movie_people_relationship_df)


     电影人ID  电影序号  关系
0     1037     1  导演
1     1037     1  主演
2     1093     1  主演
3     1037     2  导演
4     1037     2  主演
..     ...   ...  ..
334   1076    98  主演
335   1149    99  导演
336   1166    99  导演
337   1197   100  导演
338   1087   100  主演

[339 rows x 3 columns]


In [6]:
print(movie_people_relationship_df['关系'].value_counts())

主演    210
导演    129
Name: 关系, dtype: int64


In [9]:
# 创建表结构的SQL语句
create_table_movie_person = '''
Create Table movie_person_info
( 
 movie_person_id varchar(10) Not Null,
 movie_person_name varchar(20) Not Null,
 country varchar(20),
 primary key(movie_person_id)
);
'''
# gender varchar(2),

# 生成INSERT语句
insert_movie_people_statements = [create_table_movie_person]
for index, row in movie_people_df.iterrows():
    movie_person_id = row['电影人ID']
    movie_person_name = row['电影人名称']
    # 假设性别和国籍未知，用'未知'占位
#     gender = '未知'
#     country = '未知'
    # 判断电影人名称中是否包含'·'来决定国籍
    country = '美国' if '·' in movie_person_name else '中国'
    
    insert_movie_people_sql = f"insert into movie_person_info values('{movie_person_id}','{movie_person_name}','{country}');"
    insert_movie_people_statements.append(insert_movie_people_sql)
    
print(insert_movie_people_statements[0:5])
# # 打印生成的SQL语句
# for sql in insert_statements:
#     print(sql)


['\nCreate Table movie_person_info\n( \n movie_person_id varchar(10) Not Null,\n movie_person_name varchar(20) Not Null,\n country varchar(20),\n primary key(movie_person_id)\n);\n', "insert into movie_person_info values('1001','D.J.卡卢索','中国');", "insert into movie_person_info values('1002','F·加里·格雷','美国');", "insert into movie_person_info values('1003','乌尔善','中国');", "insert into movie_person_info values('1004','乔·德特杜巴','美国');"]


In [11]:
# 创建表结构的 SQL 语句
create_table_relation = '''
Create Table movie_actor_relation
( 
 id varchar(10) Not Null,
 movie_id varchar(10) Not Null,
 actor_id varchar(10) Not Null,
 relation_type varchar(20),
 primary key(id),
 foreign Key(movie_id) references movie_info(movie_id),
 foreign Key(actor_id) references actor_info(actor_id)
);
'''

# 生成 INSERT 语句
insert_relation_statements = [create_table_relation]
for index, row in movie_people_relationship_df.iterrows():
    # 生成唯一的 id（这里简单使用索引）
    id = str(index + 1)
    movie_id = str(row['电影序号'])
    actor_id = row['电影人ID']
    relation_type = row['关系']
    
    insert_relation_sql = f"insert into movie_actor_relation values('{id}','{movie_id}','{actor_id}','{relation_type}');"
    insert_relation_statements.append(insert_relation_sql)

print(insert_relation_statements[0:5])

['\nCreate Table movie_actor_relation\n( \n id varchar(10) Not Null,\n movie_id varchar(10) Not Null,\n actor_id varchar(10) Not Null,\n relation_type varchar(20),\n primary key(id),\n foreign Key(movie_id) references movie_info(movie_id),\n foreign Key(actor_id) references actor_info(actor_id)\n);\n', "insert into movie_actor_relation values('1','1','1037','导演');", "insert into movie_actor_relation values('2','1','1037','主演');", "insert into movie_actor_relation values('3','1','1093','主演');", "insert into movie_actor_relation values('4','2','1037','导演');"]


In [14]:
# 定义初始 SQL 语句
initial_statements = [
    "Create Database movie100;\n",
    "go\n",
    "use movie100;\n"
]

# 将所有语句合并为一个列表
all_statements = initial_statements + insert_movie_info_statements + insert_movie_people_statements + insert_relation_statements

# 将所有 SQL 语句写入文本文件
file_path = 'movie100.txt'
with open(file_path, 'w') as file:
    for statement in all_statements:
        file.write(statement + '\n')

###  完成了电影基本信息和电影人关系的SQL语句写入，后续有条件可以增加类型type和对应关系

In [12]:
import pandas as pd
import numpy as np

# 加载数据集
file_path = 'movieMY.xlsx'  # 替换为您的文件路径
label_data = pd.read_excel(file_path, sheet_name='标签')

In [13]:
label_data

,序号,片名,剧情,喜剧,动作,冒险,科幻,历史,战争,动画,奇幻,灾难,悬疑,惊悚,犯罪,爱情,古装,家庭,传记,青春
0,1,长津湖,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,战狼2,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,你好，李焕英,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,哪吒之魔童降世,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,流浪地球,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,哥斯拉2:怪兽之王,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,97,超时空同居,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
97,98,老炮儿,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,99,阿丽塔：战斗天使,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
labels = list(label_data.columns[2:])
label_id_name_table = pd.DataFrame({"label_id": range(1, len(labels) + 1), "label_name": labels})
label_id_name_table

,label_id,label_name
0,1,剧情
1,2,喜剧
2,3,动作
3,4,冒险
4,5,科幻
5,6,历史
6,7,战争
7,8,动画
8,9,奇幻
9,10,灾难


In [18]:
# Melting the dataframe to create a long format dataframe for movie ID and label relationships
movie_label_relationship = pd.melt(label_data, id_vars=['序号', '片名'], value_vars=labels, var_name='label_name', value_name='value')
# Filtering out rows where the label is not applicable (i.e., value is 0)
movie_label_relationship = movie_label_relationship[movie_label_relationship['value'] == 1]
# Mapping label names to their IDs
movie_label_relationship = movie_label_relationship.merge(label_id_name_table, on='label_name', how='left')
# Selecting only the necessary columns and converting movie_id to string
movie_label_relationship['序号'] = movie_label_relationship['序号'].astype(str)
movie_label_relationship = movie_label_relationship[['序号', 'label_id']]

movie_label_relationship

,序号,label_id
0,1,1
1,3,1
2,8,1
3,11,1
4,12,1
...,...,...
261,66,16
262,14,17
263,59,17
264,70,17


In [20]:
create_table_label = '''
CREATE TABLE label_table (
    label_id INT PRIMARY KEY,
    label_name VARCHAR(30)
);
'''
create_table_movie_label = '''
CREATE TABLE movie_label_relation (
    movie_id VARCHAR(10),
    label_id INT,
    PRIMARY KEY (movie_id, label_id),
    FOREIGN KEY (label_id) REFERENCES label_table(label_id),
    FOREIGN KEY (movie_id) REFERENCES movie_info(movie_id)
);
'''


insert_statements_label = "INSERT INTO label_table (label_id, label_name) VALUES "
insert_statements_label += ", ".join([f"({row.label_id}, '{row.label_name}')" for index, row in label_id_name_table.iterrows()])


insert_statements_movie_label = "INSERT INTO movie_label_relation (movie_id, label_id) VALUES "
insert_statements_movie_label += ", ".join([f"('{row['序号']}', {row.label_id})" for index, row in movie_label_relationship.iterrows()])


sql_statements = create_table_label + "\n\n" + create_table_movie_label + "\n\n" + insert_statements_label + ";\n\n" + insert_statements_movie_label + ";"


# txtfile_path = 'movie_label_relations.txt'
# with open(txtfile_path, 'w') as file:
#     file.write(sql_statements)

# print(f"SQL statements saved to {txtfile_path}")



In [21]:
# Writing to a text file
txtfile_path = 'movie100.txt'
with open(txtfile_path, 'a+') as file:
    # 将光标移到文件末尾，以防止覆盖已有内容
    file.seek(0, 2)
    
    # 写入新的 SQL 语句
    file.write(sql_statements)

print(f"SQL statements saved to {txtfile_path}")

SQL statements saved to movie100.txt
